In [1]:
%matplotlib inline

import os
import glob

import pandas as pd
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('talk')
sns.set_style('whitegrid')

obs_color = "#3498db"
hist_color = "#34495e"
rcp_color = "#e74c3c"

In [2]:
lookup = pd.read_csv('/glade/p/ral/hap/mizukami/loca/GF-loca12k/output/selected/gaugeII/gageIInat.csv')
lookup = lookup.set_index('STAID')
lookup.head()

,seg_id2,HUC02,STATE,DRAIN_SQKM,LAT_GAGE,LNG_GAGE,STANAME
STAID,,,,,,,
1030500,1000015,1,ME,3676.1720,45.500975,-68.305956,Mattawamkeag River near Mattawamkeag ME
1013500,1000048,1,ME,2252.6960,47.237394,-68.582642,Fish River near Fort Kent ME
1031500,1000241,1,ME,769.0482,45.175008,-69.314697,Piscataquis River near Dover-Foxcroft ME
1022500,1000331,1,ME,573.6006,44.607972,-67.935242,Narraguagus River at Cherryfield ME
1047000,1000399,1,ME,909.0972,44.869200,-69.955103,Carrabassett River near North Anson ME


In [3]:
huc_mapping = {'1': '1-2', '2': '1-2',
               '3': '3',
               '4': '4-6', '5': '4-6', '6': '4-6',
               # ...
               '9': '7-10',
               '10L': '7-10', '10U': '7-10',
               '11': '11-16', '12': '11-16', '13': '11-16', '14': '11-16', '15': '11-16', '16': '11-16',
               '17': '17', '18': '18'}

def get_bcsd_df(seg, huc=None):
    if huc is None:
        huc_dir = '*'
    else:
        huc_dir = huc_mapping[huc]
        
    pattern = f'/glade/p/ral/hap/mizukami/hydro_cmip5/vic/route/mizuRoute/route/output/selected/gageIInat/{huc_dir}/*.routed.selected.nc'
    print(pattern, flush=True)
    files = glob.glob(pattern)
    index = xr.Variable('ens_member', ['_'.join(os.path.basename(f).split('.')[:2]) for f in files])
    ds = xr.open_mfdataset(files, concat_dim=index, data_vars='different')
    ds = ds.rename({'reachID': 'sSeg'}).set_coords('sSeg')

    df = ds.sel(sSeg=seg)['routedRunoff'].to_dataframe()
    df = df.drop(columns='sSeg')['routedRunoff'].unstack(level=0)
    return df

In [4]:
def get_loca_df(seg):
    files = glob.glob('/glade/p/ral/hap/mizukami/loca/GF-loca12k/output/selected/gaugeII/*.routed.selected.nc')
    index = xr.Variable('ens_member', ['_'.join(os.path.basename(f).split('.')[:2]) for f in files])
    ds = xr.open_mfdataset(files, concat_dim=index, data_vars='different')
    ds = ds.rename({'reachID': 'sSeg'}).set_coords('sSeg')

    df = ds.sel(sSeg=seg)['KWTroutedRunoff'].to_dataframe()
    df = df.drop(columns='sSeg')['KWTroutedRunoff'].unstack(level=0)
    return df

In [6]:
# def get_maurer_df(seg):
    
    
# def get_livneh_df(seg):
    
    

In [5]:
def get_obs(siteno):
    strnum = str(siteno).zfill(9)
    df = pd.read_table(f'/glade/work/mizukami/data/streamflow_obs/{strnum}_streamflow_1980_leap.txt',
                       sep=r"\s*", header=None, names=['siteno', 'year', 'month', 'day', 'streamflow']).reset_index()
    
    df.index = pd.to_datetime({'year': df.year, 'month': df.month, 'day': df.day})
    df = df.drop(columns=['index', 'siteno', 'year', 'month', 'day'])
    return df * 0.028


In [8]:
# df_obs = get_obs(12451000)
# df_bcsd = get_bcsd_df(17000507)
# df_loca = get_loca_df(17000507)

In [6]:
def plotting_positions(n, alpha=0.4, beta=0.4):
    '''Returns a monotonic array of plotting positions.
    Parameters
    ----------
    n : int
        Length of plotting positions to return.
    alpha, beta : float
        Plotting positions parameter. Default is 0.4.
    Returns
    -------
    positions : ndarray
        Quantile mapped data with shape from `input_data` and probability
            distribution from `data_to_match`.
    See Also
    --------
    scipy.stats.mstats.plotting_positions
    '''
    return (np.arange(1, n + 1) - alpha) / (n + 1. - alpha - beta)

In [7]:
def make_fdc_data(df):
    pps = plotting_positions(len(df))
    vals = np.sort(df.values, axis=0)[::-1]
    return pd.DataFrame(data=vals, index=pps, columns=df.columns)

In [8]:
def make_figure_1(outfilename=None, name=None):

    fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(12, 4))
    # Why not displaying?
    
    for i, (df, label) in enumerate([(df_bcsd, 'BCSD'), (df_loca, 'LOCA')]):
        plt.sca(axes[i])

        hist = make_fdc_data(df['1970': '1999'].filter(regex='rcp85'))
        rcp45 = make_fdc_data(df['2070': '2099'].filter(regex='rcp45'))
        rcp85 = make_fdc_data(df['2070': '2099'].filter(regex='rcp85'))

        lines0 = plt.plot(hist.index, hist.values, color=hist_color, lw=0.5, alpha=0.5, zorder=10)
#         lines1 = plt.plot(rcp45.index, rcp45.values, color=rcp_color, lw=0.5, alpha=0.5)
        lines2 = plt.plot(rcp85.index, rcp85.values, color=rcp_color, lw=0.5, alpha=0.5)
        plt.yscale('log')
        plt.ylim(ymax=np.percentile(df['1970': '1999'], 99.999) * 1.1, ymin=np.percentile(df['1970': '1999'], 0.0001) * .9)
        plt.xlabel('Exceedance probability')
        plt.ylabel('Streamflow ($m^3 s^{-1})$')
        plt.title(label)

    plt.figlegend((lines0[0], lines2[0], ), ('Historical', 'RCP 8.5'))
        
    
    if name:
        fig.suptitle(name, y=1.02)
    
    if outfilename is not None:
        fig.savefig(outfilename, bbox_inches='tight', dpi=300)
        
       

In [13]:
# def make_figure_2(outfilename=None, gaugename=None):
#     fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(12, 4))

#     for i, (df, label) in enumerate([(df_bcsd, 'BCSD'), (df_loca, 'LOCA')]):
#         plt.sca(axes[i])

#         hist = make_fdc_data(df['1970': '1999'].filter(regex='rcp85'))
#         rcp45 = make_fdc_data(df['2070': '2099'].filter(regex='rcp45'))
#         rcp85 = make_fdc_data(df['2070': '2099'].filter(regex='rcp85'))

#         lines0 = plt.plot(hist.index, hist.mean(axis=1).values, color=hist_color, lw=1, zorder=10)
# #         lines1 = plt.plot(rcp45.index, rcp45.mean(axis=1).values, color=rcp_color, lw=1)
#         lines2 = plt.plot(rcp85.index, rcp85.mean(axis=1).values, color=rcp_color, lw=1)
#         plt.yscale('log')
#         plt.ylim(ymax=np.percentile(df['1970': '1999'], 99.999) * 1.1, ymin=np.percentile(df['1970': '1999'], 0.0001) * .9)
#         plt.xlabel('Exceedance probability')
#         plt.ylabel('Streamflow ($m^3 s^{-1})$')
#         plt.title(label)

#     plt.figlegend((lines0[0], lines2[0], ), ('Historical', 'RCP 8.5'), loc='center right')
    
#     if gaugename:
#         plt.title(gaugename)
    
#     if outfilename is not None:
#         fig.savefig(outfilename)

In [9]:
def make_figure_2(outfilename=None, name=None):

    fig, axes = plt.subplots(ncols=1, sharex=True, sharey=True, figsize=(12, 8))

    lines = []
    labels = []

    plt.sca(axes)
    obs = make_fdc_data(df_obs['1970': '1999'])
    obs_line = plt.plot(obs.index, obs.mean(axis=1).values, color=obs_color, lw=4, zorder=10, ls='-',)
    lines.append(obs_line[0])
    labels.append('Observations')

    for i, (df, label) in enumerate([(df_bcsd, 'BCSD'), (df_loca, 'LOCA')]):

        if i == 1:
            ls = '--'
        else:
            ls = '-'

        hist = make_fdc_data(df['1970': '1999'].filter(regex='rcp85'))
        rcp85 = make_fdc_data(df['2070': '2099'].filter(regex='rcp85'))

        lines0 = plt.plot(hist.index, hist.mean(axis=1).values, color=hist_color, lw=2, zorder=10, ls=ls)
    #     lines1 = plt.plot(rcp45.index, rcp45.mean(axis=1).values, color='orange', lw=2, ls=ls)
        lines2 = plt.plot(rcp85.index, rcp85.mean(axis=1).values, color=rcp_color, lw=2, ls=ls)
        plt.yscale('log')
        plt.xlabel('Exceedance probability')
        plt.ylabel('Streamflow ($m^3 s^{-1})$')

        lines.extend([lines0[0], lines2[0]])
        labels.extend([f'{label}-Historical', f'{label}-RCP 8.5'])

    plt.figlegend(lines, labels, loc='upper right')

    if name:
        plt.title(name)
    
    if outfilename is not None:
        fig.savefig(outfilename, bbox_inches='tight', dpi=300)


In [ ]:
bad_ids = []
for staid, row in lookup[lookup.LNG_GAGE < -100].iterrows(): 
    try:
        df_bcsd = get_bcsd_df(row.seg_id2, huc=row.HUC02)
        df_obs = get_obs(staid)
        df_loca = get_loca_df(row.seg_id2)

    except Exception as e:
        bad_ids.append(staid)
        print('skipping %s' % staid)
        print(e)
        continue
    
    make_figure_1(outfilename=f'/glade/u/home/jvano/workdir/loca_figs/fdc_ensemble_{staid}.png', name=row.STANAME)
    make_figure_2(outfilename=f'/glade/u/home/jvano/workdir/loca_figs/fdc_ensemble_avg_{staid}.png', name=row.STANAME)
#     make_figure_1(outfilename=f'/glade/u/home/jhamman/workdir/loca_figs/fdc_ensemble_{staid}.png', name=row.STANAME)
#     make_figure_2(outfilename=f'/glade/u/home/jhamman/workdir/loca_figs/fdc_ensemble_avg_{staid}.png', name=row.STANAME)